# Sliding Windows

Many applications require computations on sliding windows of streams. A sliding window is specified by a window size and a step size, both of which are positive integers. Sliding windows generate a sequence of lists from a stream. For a stream x, the the n-th window is the list x[n*step_size: n*step_size + window_size]. For example if the step_size is 4 and the window size is 2, the sequence of windows is x[0:2], x[4:6], x[8:10], ...

**map_window** applies a specified function to each window of a stream and the returned value is an element of the ouput stream

**map_window(func, in_stream, out_stream, window_size, step_size, state=None, name=None, **kwargs)**

In the next example, *y[n] = sum(x[3*n: 3*n + 2])*

In [38]:
import inspect

def f(x, y=0, *args, **kwargs):
    x = y

sig = signature(f)
print (sig)

print (sig.parameters.items())

lst = {k: v.default for k, v in sig.parameters.items() if v.default is not inspect.Parameter.empty}

print (lst)

(x, y=0, *args, **kwargs)
odict_items([('x', <Parameter "x">), ('y', <Parameter "y=0">), ('args', <Parameter "*args">), ('kwargs', <Parameter "**kwargs">)])
{'y': 0}


In [1]:
import os
import sys
sys.path.append("../")

from IoTPy.core.stream import Stream, run
from IoTPy.agent_types.op import map_window
from IoTPy.helper_functions.recent_values import recent_values

def example():
    x, y = Stream(), Stream()

    map_window(func=sum, in_stream=x, out_stream=y, window_size=2, step_size=3)

    x.extend([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[1, 7, 13, 19]


## Sliding Windows with State and Keyword Arguments

You can specify a state using the keyword **state** and giving the state an initial value. The function can also use keyword arguments as illustrated in the following example in which the keyword is *threshold*.

This example computes *mean_of_this_window* which is the mean of the current window, and it computes *max_of_window_mean* which is the maxium of *mean_of_this_window* over all windows seen so far. It computes *deviation* which is the deviation of the mean of the current window from the max seen so far, and it sets the deviation to 0.0 if it is below a threshold.

In [16]:
import numpy as np

def example():
    
    def deviation_from_max(window, max_of_window_mean, threshold):
        mean_of_this_window = np.mean(window)
        max_of_window_mean = max(max_of_window_mean, mean_of_this_window)
        deviation = max_of_window_mean - mean_of_this_window
        if deviation < threshold: deviation = 0.0
        return deviation, max_of_window_mean
        
        
    x, y = Stream(), Stream()

    map_window(func=deviation_from_max, in_stream=x, out_stream=y, 
               window_size=2, step_size=1, state=0, threshold=4)

    x.extend([0, 10, 2, 4, 0, 40, 20, 4])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 18.0]


In [39]:
from IoTPy.agent_types.op import timed_window
def example():
    def f(input_list):
        print ('input_list ', input_list)
        total = sum([v[1] for v in input_list])
        print ('total ', total)
        return []
    
    x, y = Stream(), Stream()
    
    timed_window(f, x, y, window_duration=4, step_time=4)
    
    x.extend([[1, 100], [3, 250], [5, 400], [5.5, 100], [7, 300], [11.0, 250]])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()
    

input_list  [[1, 100], [3, 250]]
total  350
input_list  [[5, 400], [5.5, 100], [7, 300]]
total  800
recent values of stream y are
[(4, []), (8, [])]
